In [3]:
%pip install azure-ai-ml
# %pip install -U 'azureml-rag[faiss]>=0.1.11'
%pip install azureml-rag
# If using hugging_face embeddings add `hugging_face` extra, e.g. `azureml-rag[faiss,hugging_face]`

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: C:\Users\h9511\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: C:\Users\h9511\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Create a FAISS based Vector Index for Document Retrieval with AzureML

We'll walk through setting up an AzureML Pipeline which pulls a Git Repo, processes the data into chunks, embeds the chunks, and creates a LangChain-compatible FAISS Vector Index.

## Get client for AzureML Workspace

The workspace is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

If you don't have a Workspace and want to create and Index locally see [here to create one](https://learn.microsoft.com/azure/machine-learning/quickstart-create-resources?view=azureml-api-2).

Enter your Workspace details below, running this still will write a `workspace.json` file to the current folder.

In [5]:
%%writefile workspace.json
{
    "subscription_id": "2d3e41fb-7962-46d1-8dcd-c6d70547e2d1",
    "resource_group": "RG-EA-DATA",
    "workspace_name": "ea-data-intern-ml",
    "target": "https://ea-data-intern-openai.openai.azure.com/",
    "api_key": "8b8dde791c4242639de3c11d053e8efb"
}

Overwriting workspace.json


`MLClient` is how you interact with AzureML

In [6]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azureml.core import Workspace

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

try:
    ml_client = MLClient.from_config(credential=credential, path="workspace.json")
except Exception as ex:
    raise Exception(
        "Failed to create MLClient from config file. Please modify and then run the above cell with your AzureML Workspace details."
    ) from ex
    # ml_client = MLClient(
    #     credential=credential,
    #     subscription_id="",
    #     resource_group_name="",
    #     workspace_name=""
    # )

ws = Workspace(
    subscription_id=ml_client.subscription_id,
    resource_group=ml_client.resource_group_name,
    workspace_name=ml_client.workspace_name,
)
print(ml_client)

DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot this issue.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint.
	SharedTokenCacheCredential: SharedTokenCacheCredential authentication unavailable. No accounts were found in the cache.
	AzureCliCredential: Azure CLI not found on path
	AzurePowerShellCredential: Az.Account module >= 2.2.0 is not installed
	AzureDeveloperCliCredential: Azure Developer CLI could not be found. Please visit https://aka.ms/azure-dev for installation instructions and then,once installed, authenticate to your Azure account using 'azd auth login'.
To mitigate this issue, please refer to the troubleshooting guidelines here at https:/

Performing interactive authentication. Please follow the instructions on the terminal.


The default web browser has been opened at https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.


KeyboardInterrupt: 

## Which Datasource?

For demo purposes a public repo is being used, though we've included steps at the end of the notebook to use username + password for a private repository (the secrets are stored in an Azure KeyVault and only access at Runtime on the Compute doing the git clone).


In [7]:
git_url = "https://github.com/rust-lang/book.git"

## Which Embeddings Model to use?

There are currently two supported Embedding options: OpenAI's `text-embedding-ada-002` embedding model or HuggingFace embedding models. Here are some factors that might influence your decision:

### OpenAI

OpenAI has [great documentation](https://platform.openai.com/docs/guides/embeddings) on their Embeddings model `text-embedding-ada-002`, it can handle up to 8191 tokens and can be accessed using [Azure OpenAI](https://learn.microsoft.com/azure/cognitive-services/openai/concepts/models#embeddings-models) or OpenAI directly.
If you have an existing Azure OpenAI Instance you can connect it to AzureML, if you don't AzureML provisions a default one for you called `Default_AzureOpenAI`.
The main limitation when using `text-embedding-ada-002` is cost/quota available for the model. Otherwise it provides high quality embeddings across a wide array of text domains while being simple to use.

### HuggingFace

HuggingFace hosts many different models capable of embedding text into single-dimensional vectors. The [MTEB Leaderboard](https://huggingface.co/spaces/mteb/leaderboard) ranks the performance of embeddings models on a few axis, not all models ranked can be run locally (e.g. `text-embedding-ada-002` is on the list), though many can and there is a range of larger and smaller models. When embedding with HuggingFace the model is loaded locally for inference, this will potentially impact your choice of compute resources.

**NOTE:** The default PromptFlow Runtime does not come with HuggingFace model dependencies installed, Indexes created using HuggingFace embeddings will not work in PromptFlow by default. **Pick OpenAI if you want to use PromptFlow**

### Run the cells under _either_ heading (OpenAI or HuggingFace) to use the respective embedding model

#### OpenAI

We can use the automatically created `Default_AzureOpenAI` connection.

If you would rather use an existing Azure OpenAI connection then change `aoai_connection_name` below.
If you would rather use an existing Azure OpenAI resource, but don't have a connection created, modify `aoai_connection_name` and the details under the `# Create New Connection` code comment, or navigate the PromptFlow section in your AzureML Workspace and use the Connections create UI flow.

In [8]:
aoai_connection_name = "Default_AzureOpenAI"
aoai_connection_id = None

In [10]:
%pip install --upgrade azure-ai-ml

   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.5 MB 991.0 kB/s eta 0:00:11
   ---------------------------------------- 0.1/10.5 MB 825.8 kB/s eta 0:00:13
   ---------------------------------------- 0.1/10.5 MB 825.8 kB/s eta 0:00:13
   ---------------------------------------- 0.1/10.5 MB 654.9 kB/s eta 0:00:16
    --------------------------------------- 0.2/10.5 MB 841.6 kB/s eta 0:00:13
    --------------------------------------- 0.2/10.5 MB 841.6 kB/s eta 0:00:13
   - -------------------------------------- 0.3/10.5 MB 905.4 kB/s eta 0:00:12
   - -------------------------------------- 0.5/10.5 MB 1.2 MB/s eta 0:00:09
   - -------------------------------------- 0.5/10.5 MB 1.1 MB/s eta 0:00:10
   -- ------------------------------------- 0.7/10.5 MB 1.4 MB/s eta 0:00:07
   -- ------------------------------------- 0.7/10.5 MB 1.4 MB/s eta 0:00:07
   --- ------------------------------------ 0.8/10.5 MB 1.4 MB/s eta 


[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: C:\Users\h9511\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [13]:
%pip install azure-ai-ml==1.9.0 azureml-core==1.50.0 azureml-rag==0.1.11

Note: you may need to restart the kernel to use updated packages.Collecting azure-ai-ml==1.9.0
  Using cached pydash-5.1.2-py3-none-any.whl.metadata (42 kB)
  Using cached azureml_dataprep-4.11.7-py3-none-any.whl.metadata (2.1 kB)
  Using cached dotnetcore2-3.1.23-py3-none-win_amd64.whl.metadata (1.3 kB)
     ---------------------------------------- 0.0/873.1 kB ? eta -:--:--
     - ----------------------------------- 30.7/873.1 kB 660.6 kB/s eta 0:00:02
     - ----------------------------------- 41.0/873.1 kB 653.6 kB/s eta 0:00:02
     - ----------------------------------- 41.0/873.1 kB 653.6 kB/s eta 0:00:02
     --- --------------------------------- 71.7/873.1 kB 357.2 kB/s eta 0:00:03
     ----- ------------------------------ 122.9/873.1 kB 514.3 kB/s eta 0:00:02
     ------- ---------------------------- 174.1/873.1 kB 655.4 kB/s eta 0:00:02
     ------- ---------------------------- 174.1/873.1 kB 655.4 kB/s eta 0:00:02
     ----------- ------------------------ 276.5/873.1 kB 710.

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [227 lines of output]
  <string>:36: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  WARNING setuptools_scm.pyproject_reading toml section missing 'pyproject.toml does not contain a tool.setuptools_scm section'
  Traceback (most recent call last):
    File "C:\Users\h9511\AppData\Local\Temp\pip-build-env-xcdf0f2p\overlay\Lib\site-packages\setuptools_scm\_integration\pyproject_reading.py", line 36, in read_pyproject
      section = defn.get("tool", {})[tool_name]
                ~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^
  KeyError: 'setuptools_scm'
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-311
  creating build\lib.win-amd64-cpython-311\pyarrow
  copying pyarrow\benchmark.py -> build\lib.win-amd64-cpython-311\pyarrow
  copying pyarrow\cffi.py -> build\lib.win-amd64-cpython-311\pyarrow
  copying p


   ---------------------------------------  15.9/16.0 MB 961.4 kB/s eta 0:00:01
   ---------------------------------------  16.0/16.0 MB 961.4 kB/s eta 0:00:01
   ---------------------------------------  16.0/16.0 MB 960.0 kB/s eta 0:00:01
   ---------------------------------------  16.0/16.0 MB 960.0 kB/s eta 0:00:01
   ---------------------------------------- 16.0/16.0 MB 954.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/31.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/31.7 MB 991.0 kB/s eta 0:00:32
   ---------------------------------------- 0.1/31.7 MB 1.1 MB/s eta 0:00:31
   ---------------------------------------- 0.1/31.7 MB 1.0 MB/s eta 0:00:31
   ---------------------------------------- 0.2/31.7 MB 1.1 MB/s eta 0:00:31
   ---------------------------------------- 0.2/31.7 MB 981.9 kB/s eta 0:00:33
   ---------------------------------------- 0.3/31.7 MB 983.0 kB/s eta 0:00:33
   ---------------------------------------- 0.3/31.7 MB 999.9

In [11]:
from azureml.rag.utils.connections import (
    get_connection_by_name_v2,
    create_connection_v2,
)

try:
    aoai_connection = get_connection_by_name_v2(ws, aoai_connection_name)
except Exception as ex:
    # Create New Connection
    # Modify the details below to match the `Endpoint` and API key of your AOAI resource, these details can be found in Azure Portal
    raise RuntimeError(
        "Have you entered your AOAI resource details below? If so, delete me!"
    )
    target = "<target>"  # example: 'https://<endpoint>.openai.azure.com/'
    key = "<key>"
    apiVersion = "2023-03-15-preview"
    if key == "<key>":
        raise RuntimeError(f"Please provide a valid key for the Azure OpenAI service")
    if target == "<target>":
        raise RuntimeError(
            f"Please provide a valid target for the Azure OpenAI service"
        )
    if apiVersion == "<api_version>":
        raise RuntimeError(
            f"Please provide a valid api-version for the Azure OpenAI service"
        )
    aoai_connection_id = create_connection_v2(
        workspace=ws,
        name=aoai_connection_name,
        category="AzureOpenAI",
        target=target,
        auth_type="ApiKey",
        credentials={"key": key},
        metadata={"ApiType": "azure", "ApiVersion": apiVersion},
    )["id"]

aoai_connection_id = aoai_connection["id"]

ImportError: cannot import name 'NoneCredentialConfiguration' from 'azure.ai.ml.entities' (C:\Users\h9511\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\azure\ai\ml\entities\__init__.py)

Now that your Workspace has a connection to Azure OpenAI we will make sure the `text-embedding-ada-002` model has been deployed ready for inference. This cell will fail if there is not deployment for the embeddings model, [follow these instructions](https://learn.microsoft.com/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal#deploy-a-model) to deploy a model with Azure OpenAI.

In [ ]:
from azureml.rag.utils.deployment import infer_deployment

aoai_embedding_model_name = "text-embedding-ada-002"
try:
    aoai_embedding_deployment_name = infer_deployment(
        aoai_connection, aoai_embedding_model_name
    )
    print(
        f"Deployment name in AOAI workspace for model '{aoai_embedding_model_name}' is '{aoai_embedding_deployment_name}'"
    )
except Exception as e:
    print(f"Deployment name in AOAI workspace for model '{model_name}' is not found.")
    print(
        f"Please create a deployment for this model by following the deploy instructions on the resource page for '{aoai_connection['properties']['target']}' in Azure Portal."
    )

Finally we will combine the deployment and model information into a uri form which the AzureML embeddings components expect as input.

In [ ]:
embeddings_model_uri = f"azure_open_ai://deployment/{aoai_embedding_deployment_name}/model/{aoai_embedding_model_name}"

#### HuggingFace

AzureMLs default model from HuggingFace is `all-mpnet-base-v2`, it can be run by most laptops. Any `sentence-transformer` models should be supported, you can learn more about `sentence-transformers` [here](https://huggingface.co/sentence-transformers).

In [ ]:
embeddings_model_uri = "hugging_face://model/sentence-transformers/all-mpnet-base-v2"

## Setup Pipeline to process data into Index

AzureML [Pipelines](https://learn.microsoft.com/azure/machine-learning/concept-ml-pipelines?view=azureml-api-2) connect together multiple [Components](https://learn.microsoft.com/azure/machine-learning/concept-component?view=azureml-api-2). Each Component defines inputs, code that consumes the inputs and outputs produced from the code. Pipelines themselves can have inputs, and outputs produced by connecting together individual sub Components.
To process your data for embedding and indexing we will chain together multiple components each performing their own step of the workflow.

The Components are published to a [Registry](https://learn.microsoft.com/azure/machine-learning/how-to-manage-registries?view=azureml-api-2&tabs=cli), `azureml`, which should have access to by default, it can be accessed from any Workspace.
In the below cell we get the Component Definitions from the `azureml` registry.

In [ ]:
ml_registry = MLClient(credential=credential, registry_name="azureml")

# Clones git repository to output folder of pipeline, by default this will be on the default Workspace Datastore `workspaceblobstore`
git_clone_component = ml_registry.components.get("llm_rag_git_clone", label="latest")
# Walks input folder according to provided glob pattern (all files by default: '**/*') and attempts to open them, extract text chunks and further chunk if necessary to fir within provided `chunk_size`.
crack_and_chunk_component = ml_registry.components.get(
    "llm_rag_crack_and_chunk", label="latest"
)
# Reads input folder of files containing chunks and their metadata as batches, in parallel, and generates embeddings for each chunk. Output format is produced and loaded by `azureml.rag.embeddings.EmbeddingContainer`.
generate_embeddings_component = ml_registry.components.get(
    "llm_rag_generate_embeddings", label="latest"
)
# Reads input folder produced by `azureml.rag.embeddings.EmbeddingsContainer.save()` and inserts all documents (chunk, metadata, embedding_vector) int a Faiss index and in-memory document store. Writes an MLIndex yaml detailing the index and embeddings model information.
create_faiss_index_component = ml_registry.components.get(
    "llm_rag_create_faiss_index", label="latest"
)
# Takes a uri to a storage location where an MLIndex yaml is stored and registers it as an MLIndex Data asset in the AzureML Workspace.
register_mlindex_component = ml_registry.components.get(
    "llm_rag_register_mlindex_asset", label="latest"
)

Each Component has documentation which provides an overall description of the Components purpose and each of the inputs/outputs.
For example we can see understand what `crack_and_chunk` does by inspecting the Component definition.

In [ ]:
print(crack_and_chunk_component)

Below a Pipeline is built by defining a python function which chains together the above components inputs and outputs. Arguments to the function are inputs to the Pipeline itself and the return value is a dictionary defining the outputs of the Pipeline.

In [15]:
from azure.ai.ml import Input, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.entities._job.pipeline._io import PipelineInput
from typing import Optional


def use_automatic_compute(component, instance_count=1, instance_type="Standard_E8s_v3"):
    """Configure input `component` to use automatic compute with `instance_count` and `instance_type`.

    This avoids the need to provision a compute cluster to run the component.
    """
    component.set_resources(
        instance_count=instance_count,
        instance_type=instance_type,
        properties={"compute_specification": {"automatic": True}},
    )
    return component


def optional_pipeline_input_provided(input: Optional[PipelineInput]):
    """Checks if optional pipeline inputs are provided."""
    return input is not None and input._data is not None


# If you have an existing compute cluster you want to use instead of automatic compute, uncomment the following line, replace `dedicated_cpu_compute` with the name of your cluster.
# Also comment out the `component.set_resources` line in `use_automatic_compute` above and the `default_compute='serverless'` line below.
# @pipeline(compute=dedicated_cpu_compute)
@pipeline(default_compute="serverless")
def git_to_faiss(
    git_url: str,
    embeddings_model: str,
    asset_name: str,
    branch_name: str = None,
    chunk_size: int = 1024,
    data_source_glob: str = None,
    data_source_url: str = None,
    document_path_replacement_regex: str = None,
    git_connection_id=None,
    aoai_connection_id=None,
    embeddings_container=None,
):
    """Pipeline to generate embeddings for a `input_data` source and create a Faiss index."""

    git_clone = git_clone_component(git_repository=git_url, branch_name=branch_name)
    use_automatic_compute(git_clone)
    if optional_pipeline_input_provided(git_connection_id):
        git_clone.environment_variables[
            "AZUREML_WORKSPACE_CONNECTION_ID_GIT"
        ] = git_connection_id

    crack_and_chunk = crack_and_chunk_component(
        input_data=git_clone.outputs.output_data,
        input_glob=data_source_glob,
        chunk_size=chunk_size,
        data_source_url=data_source_url,
        document_path_replacement_regex=document_path_replacement_regex,
    )
    use_automatic_compute(crack_and_chunk)

    generate_embeddings = generate_embeddings_component(
        chunks_source=crack_and_chunk.outputs.output_chunks,
        embeddings_container=embeddings_container,
        embeddings_model=embeddings_model,
    )
    use_automatic_compute(generate_embeddings)
    if optional_pipeline_input_provided(aoai_connection_id):
        generate_embeddings.environment_variables[
            "AZUREML_WORKSPACE_CONNECTION_ID_AOAI"
        ] = aoai_connection_id
    if optional_pipeline_input_provided(embeddings_container):
        # If provided, `embeddings_container` is expected to be a URI to folder, the folder can be empty.
        # Each sub-folder is generated by a `create_embeddings_component` run and can be reused for subsequent embeddings runs.
        generate_embeddings.outputs.embeddings = Output(
            type="uri_folder", path=f"{embeddings_container.path}/{{name}}"
        )

    create_faiss_index = create_faiss_index_component(
        embeddings=generate_embeddings.outputs.embeddings,
    )
    use_automatic_compute(create_faiss_index)

    register_mlindex = register_mlindex_component(
        storage_uri=create_faiss_index.outputs.index, asset_name=asset_name
    )
    use_automatic_compute(register_mlindex)
    return {
        "mlindex_asset_uri": create_faiss_index.outputs.index,
        "mlindex_asset_id": register_mlindex.outputs.asset_id,
    }

ImportError: cannot import name 'is_bytecode_optimization_enabled' from 'azure.ai.ml._utils.utils' (C:\Users\h9511\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\azure\ai\ml\_utils\utils.py)

Now we can create the Pipeline Job by calling the `@pipeline` annotated function and providing input arguments.
`asset_name` will be used when registering the MLIndex Data Asset produced by the `register_mlindex` component in the pipeline. This is how you can refer to the MLIndex within AzureML.

`crack_and_chunk` will infer the data_source_url if the input_data is a git repository. If you want to have source links your index that link to somewhere else, refer to how `data_source_url` is handled in [ACS MLIndex Notebook](../azure_cognitive_search/acs_mlindex_with_langchain.ipynb).

In [ ]:
asset_name = "rustdocs_aoai_faiss_mlindex"
data_source_glob = "src/**/*"

In [ ]:
from azure.ai.ml import Input

pipeline_job = git_to_faiss(
    git_url=git_url,
    data_source_glob=data_source_glob,
    # Each run will save latest Embeddings to subfolder under this path, runs will load latest embeddings from container and reuse any unchanged chunk embeddings.
    embeddings_model=embeddings_model_uri,
    aoai_connection_id=aoai_connection_id,
    embeddings_container=Input(
        type="uri_folder",
        path=f"azureml://datastores/workspaceblobstore/paths/embeddings/{asset_name}",
    ),
    # Name of asset to register MLIndex under
    asset_name=asset_name,
)

# By default AzureML Pipelines will reuse the output of previous component Runs when inputs have not changed.
# If you want to rerun the Pipeline every time each time so that any changes to upstream data sources are processed uncomment the below line.
# pipeline_job.settings.force_rerun = True # Rerun each time so that git_clone isn't cached, if intent is to ingest latest data.

Finally we add some properties to `pipeline_job` which ensure the Index generation progress and final Artifact appear in the PromptFlow Vector Index UI.

In [ ]:
# These are added so that in progress index generations can be listed in UI, this tagging is done automatically by UI.
pipeline_job.properties["azureml.mlIndexAssetName"] = asset_name
pipeline_job.properties["azureml.mlIndexAssetKind"] = "faiss"
pipeline_job.properties["azureml.mlIndexAssetSource"] = "Git Repository"

## Submit Pipeline

**In case of any errors see [TROUBLESHOOT.md](../../TROUBLESHOOT.md).**

The output of each step in the pipeline can be inspected via the Workspace UI, click the link under 'Details Page' after running the below cell. 

In [ ]:
running_pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="git_to_faiss"
)
running_pipeline_job

In [ ]:
ml_client.jobs.stream(running_pipeline_job.name)

## Use Index with langchain

The Data Asset produced by the AzureML Pipeline above contains a yaml file named 'MLIndex' which contains all the information needed to use the Azure Cognitive Search index.
For instance if an AOAI deployment was used to embed the documents the details of that deployment and a reference to the secret are there.
This allows easy loading of the MLIndex into a langchain retriever.

In [ ]:
from azureml.rag.mlindex import MLIndex

question = "What makes Rust Enums so awesome?"

retriever = MLIndex(
    ml_client.data.get(asset_name, label="latest")
).as_langchain_retriever()
retriever.get_relevant_documents(question)

If you have not deployed `gpt-35-turbo` on your Azure OpenAI resource the below cell will fail indicated the `API deployment for this resource does not exist`. Follow the previous instructions for deploying `text-embedding-ada-002` to deploy `gpt-35-turbo`, note the chosen deployment name below and use the same or update it if you choose different one.

In [ ]:
from langchain.chains import RetrievalQA
from azureml.rag.models import init_llm, parse_model_uri

model_config = parse_model_uri(
    "azure_open_ai://deployment/gpt-35-turbo/model/gpt-35-turbo"
)
model_config["api_base"] = aoai_connection["properties"]["target"]
model_config["key"] = aoai_connection["properties"]["credentials"]["key"]
model_config["temperature"] = 0.3
model_config["max_retries"] = 3

qa = RetrievalQA.from_chain_type(
    llm=init_llm(model_config), chain_type="stuff", retriever=retriever
)

qa.run(question)

Note: To control the number of documents returned when searching try getting the the MLIndex `as_langchain_vectorstore()` instead, this implements the `VectorStore` interface which has more parameters.

## Use MLIndex with PromptFlow

To use the MLindex in PromptFlow the asset_id can be used with the `Vector Index Lookup​` Tool. Replace `versions/2` with `versions/latest` to use the latest version.

In [ ]:
asset_id = f"azureml:/{ml_client.data.get(asset_name, label='latest').id}"
asset_id.replace("resourceGroups", "resourcegroups")

## [Optional] Use Private Git Repo

By registering a connection that stores a git repository username and password we can leverage the same pipeline on private data. 

In [ ]:
git_connection_name = "github-PrivateRepo"

In [ ]:
from azureml.rag.utils.connections import (
    get_connection_by_name_v2,
    create_connection_v2,
)

try:
    git_connection = get_connection_by_name_v2(ws, git_connection)
except Exception as ex:
    print(f"Could not get connection '{git_connection}', creating a new one")
    git_pat = "<pat>"
    username = "<username>"
    if git_pat == "<pat>":
        raise RuntimeError(f"Please provide a valid PAT for the git repo")
    if username == "<username>":
        raise RuntimeError(f"Please provide a valid username for the git repo")
    git_connection = create_connection_v2(
        workspace=ws,
        name=git_connection_name,
        category="Git",
        target=git_url,
        auth_type="PAT",
        credentials={
            "pat": git_pat,
        },
        metadata={"username": username},
    )["id"]

git_connection_id = git_connection["id"]

In [ ]:
from azure.ai.ml import Input
import json

pipeline_job = git_to_faiss(
    git_url=git_url,
    git_connection_id=git_connection_id,
    data_source_glob=data_source_glob,
    # Each run will save latest Embeddings to subfolder under this path, runs will load latest embeddings from container and reuse any unchanged chunk embeddings
    embeddings_container=Input(
        type="uri_folder",
        path=f"azureml://datastores/workspaceblobstore/paths/embeddings/{asset_name}",
    ),
    embeddings_model=embeddings_model_uri,
    aoai_connection_id=aoai_connection_id,
    # Name of asset to register MLIndex under
    asset_name=asset_name,
)

# pipeline_job.settings.force_rerun = True # Rerun each time so that git_clone isn't cached, if intent is to ingest latest data.

Now go back up to the `pipeline_job.properties` cell above `Submit Pipeline` and run them to submit new pipeline using private git repo.